# Basics

(on language constructs to create hardware)

A description of a digital hardware circuit is a priori event sensitive, i.e. if a source signal changes, it will propagate. We therefore have to deal with a `Signal` data type. This is a wrapper for any other, possibly built-in wire type, like a bool.

For integer types with defined bit length, a new class is required that emulates integer calculations up to some point. This is covered by the `intbv` integer bit vector class.

**Note** The intbv class name was taken from its MyHDL origin. However, it is not entirely compatible to it and is in fact based on a redesigned emulation handled by the internal `sig` kernel..

Once a cyrite Signal type is combined with a binary operator such as '+', it will return a convertible expression type, not the actual result. In fact, it will create a logic element immediately. Let's examine this:

In [1]:
from cyhdl import *

a, b = [ Signal(intbv(0x1f0)[9:], name = n) for n in "ab" ]

In [2]:
expr_add = a + b

In [3]:
expr_add

ADD(a, b)

To get at the actual result, the expression must be evaluated:

In [4]:
expr_add.evaluate()

992

An expression by itself does not really create any logic yet, because it is not connected. Once we assign it to a signal usingn `.set`, we have a synthesizeable instance:

In [5]:
action = a.set(expr_add)

We can evaluate this action as well, but it may go wrong:

In [6]:
try:
    action.evaluate()
except Exception as e:
    print("Failure:", e)

Failure: intbv value 992 >= maximum 512


Evaluation will actually assign the value to the internal wire value. Repeated evaluation of the above will thus increase the value, but always report an error.

Two things happened here:
   * Signal `a` is driven, it is here driver and source, simultaneosly
   * An abstract adder logic was created, that can result in a synthesizeable hardware element, or it can be simulated.

Now here's a catch: `a` is Source and Driver in the same expression. This is *combinatorial loop*. If this was instanced into a hardware element, such a loop would produce new values at high frequencies and a delay would come into play.

Such a construct is therefore only valid inside a clocked construction, where the value is assigned at the event of a rising or falling edge, not at the change of the source a.

We can import such a Flip flop from an example library. It is instanced like a function with the result of a synthesizeable expression.

In [7]:
from cyrite.examples.libprimitives import MyPrimitives

m = MyPrimitives("")
ff = m.SFF

However, the `ff` is context sensitive and its instancing process implicitely checks the parenting hierarchy. Therefore we have to pack it into a hardware unit with port inputs and outputs.
We also need to create a specific clock signal instance.

In [8]:
from myirl.kernel.sensitivity import genprocess
clk = ClkSignal()

@block
def worker(clk : ClkSignal, a, b):
    
    action = a.set(ff(clk, a + b))
    
    return  [ action ]

Then we create an instance of this hardware block:

In [9]:
uut = worker(clk, a, b)

 DEBUG Inline blackbox instance [_inline_component 'SFF/SFF'] 


Now there's a problem: This will actually refuse to transfer to synthesis. Why is that? The `@block` design rules require well defined in- or outputs. However, Signal `a` is driver and source at the same time. Also, how would we determine a start value for a? We have to introduce a 'reset' and thus need to rewrite it such that it assigns the initial value at the reset event. This would become unreadable for the above construction. Therefore we rewrite it as follows:

In [10]:
@block
def unit_add(clk : ClkSignal, reset : ResetSignal,
            a_init, a: Signal.Output, b : Signal):
    
    w = a.clone("a_internal")
    
    @always(clk.posedge)
    def worker():
        if reset:
            w.next = a_init
        else:
            w.next = w + b
            
    assignments = [
        a   @assign@   w
    ]
    
    return instances()

We make an internal copy of the resulting `a` signal. Then, the `@always` process defines the functionality in a notation differing from the above `.set` action using the MyHDL-alike `.next` assignment. Behind the curtains, this `@always` process is translated into an intermediate representation. This is explained further below.

From the parent, we define the size of `a` to have some head room:

In [11]:
a = Signal(intbv()[20:])
reset = ResetSignal(True, True)
uut = unit_add(clk, reset, 5, a, b)

At a later point, we can verify if this instance is doing the right thing. For now, we synthesize it into hardware via the `pyosys` target.

When we call the .elab method by specifying a RTLIL target instance, we get a list of design elements. We pick the first, which is always the top level design.

In [12]:
from myirl.targets import pyosys

d = uut.elab(pyosys.RTLIL("top"))
design = d[0]

 Adding module with name `unit_add` 
 FINALIZE implementation `unit_add` of `unit_add` 


### RTL display

Once the design has entered the yosys RTLIL object world, it can be displayed using the yosys internal `.dot` output. This is a rudimentary debugging method however and should not be used for large designs.

In [13]:
design.display_rtl(inline = True)

Here we can see the inferred elements. An `if..else` sequence typically creates multiplexers (`$mux`). The `clk` sensitive process again infers a `$dff` flip flop type.

Also, we note that an extension to the full 20 bits occurs before the `$add` primitive.
Once we execute the necessary mapping steps in synthesis that translate those elements into a LUT and FF target structure, they might look completely different, but should behave the same.

## A short peek into internals

The cyrite library is based on a generator kernel that creates the resulting hardware or language target elements by execution rather than AST translation. This internal representation layer is referred to as the myirl kernel. All cyrite or MyHDL alike hardware descriptions on top however are AST-translated into a generator form. An example:

### A simple multiplexer unit

Normally, you would write a behavioral model for a multiplexer as follows:

In [14]:
from cyhdl import *

@block
def mux_unit(en: Signal, a_in : Signal, b_out : Signal):
    @always(en, a_in)
    def muxer():
        if en:
            b_out.next = a_in
        else:
            b_out.next = ~a_in
    return muxer

    @sequence
    def main():
        a.next = 0

Simple top level `@block` can be displayed in their explicit IRL:

In [15]:
print(mux_unit.unparse())

Unparsing unit mux_unit


@block
def mux_unit(en: Signal, a_in: Signal, b_out: Signal):

    @always_(en, a_in)
    def muxer(_context):
        (yield [_context.If(en).Then(b_out.set(a_in)).Else(b_out.set((~ a_in)))])
    return muxer

    @generator_ctx
    def main(_context):
        (yield [a.set(0)])



To display the resulting HDL, we instance a DummyTargetModule design context. When emitting instances within that context, HDL results are sent to sys.stdout. This allows us to examine language constructs in a granular way.

In [16]:
en = Signal(bool())
s = [ Signal(intbv()[4:]) for _ in range(2) ]

from myirl.targets.dummy import DummyTargetModule
ctx = DummyTargetModule(targets.Verilog)

uut = mux_unit(en, s[0], s[1])

for inst in uut.instances:
    inst(ctx) # Call instances to within context
    inst.emit(ctx)


always @ (en or a_in) begin : MUXER
    if (en) begin
        b_out <= a_in; /* fallback */
    end else begin
        b_out <= ~a_in; /* fallback */
    end
end


Note: at this stage, no distinction between Verilog type reg and wire happens, because no signal analysis has been done throughout a hierarchy. Therefore a 'fallback' tag comment is inserted by the translation.

## Notation issues: Generator versus native execution

We note that the `.next` assignment statements are translated to `.set` methods. The reason for this is, that we use generators via `yield` in our intermediate representation language (dialect), which require us to return a convertible expression or a generator for every statement.

On the other hand, there is a reason to be able to execute a function two ways: Either by native execution ('as is') to simulate its behaviour using the Python interpreter or by running it through a generator framework to transpile to target code. If we removed the `@always` decorator, we can only run this function natively, but not generate something from it that preserves conditional expressions with full coverage as well as assigment actions, unless we used AST translation.

This dual characteristic of a generator function must be accounted for in the entire concept of CyriteHDL and its underlying IRL kernel.

Detailed follow up: [ Generators - when to use yield and when not ](generators.ipynb)